In [ ]:
import pandas as pd

# Option 1: Skip bad lines (faster, but loses data)
df = pd.read_csv(
    'amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv',
    sep='\t',
    on_bad_lines='skip',  # Skips problematic lines
    quoting=3  # Optional: QUOTE_NONE - helps if quotes are causing issues
)

print(df.head())
print(f"Loaded {len(df)} rows successfully.")

In [ ]:
print(df.columns)
print(len(df))